In [16]:
import pandas as pd

In [22]:
data = pd.read_csv('data.csv')

In [23]:
data = data.sort_values('time', ascending=False).reset_index()  # Se ordena la base de datos según fecha de terremoto
data = data.drop('index',1)

In [24]:
data

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2018-10-27T18:33:12.420Z,37.6457,20.8702,10.00,4.7,mb,NaN,57.0,2.228,0.89,...,2018-10-27T20:16:45.040Z,"8km SSE of Lithakia, Greece",earthquake,6.8,1.9,0.100,30.0,reviewed,us,us
1,2018-10-27T16:57:27.956Z,65.2234,-151.6636,16.60,5.3,ml,NaN,NaN,NaN,0.71,...,2018-10-27T23:51:01.479Z,"18km ENE of Tanana, Alaska",earthquake,NaN,0.2,NaN,NaN,reviewed,ak,ak
2,2018-10-27T15:46:43.390Z,-22.4704,-68.6516,113.79,4.7,mb,NaN,106.0,0.278,0.74,...,2018-10-27T16:05:13.040Z,"28km E of Calama, Chile",earthquake,7.1,3.8,0.159,12.0,reviewed,us,us
3,2018-10-27T15:00:46.800Z,-56.0652,-27.4276,93.78,4.9,mb,NaN,69.0,5.491,0.87,...,2018-10-27T15:44:42.040Z,"72km NNW of Visokoi Island, South Georgia and ...",earthquake,7.2,6.2,0.122,21.0,reviewed,us,us
4,2018-10-27T14:33:26.470Z,37.3128,20.4930,10.00,4.6,mb,NaN,117.0,2.457,1.26,...,2018-10-27T15:07:27.040Z,"53km SW of Lithakia, Greece",earthquake,8.0,2.0,0.316,4.0,reviewed,us,us
5,2018-10-27T12:15:32.140Z,-19.3581,169.4061,207.24,4.5,mb,NaN,143.0,3.720,0.64,...,2018-10-27T12:48:16.040Z,"25km NE of Isangel, Vanuatu",earthquake,11.1,6.2,0.074,53.0,reviewed,us,us
6,2018-10-27T12:13:15.940Z,-20.6365,-66.6667,236.32,4.8,mb,NaN,43.0,2.384,1.14,...,2018-10-27T12:32:14.040Z,"25km SE of Uyuni, Bolivia",earthquake,6.8,5.8,0.049,130.0,reviewed,us,us
7,2018-10-27T11:39:13.520Z,40.1429,142.4098,50.28,5.0,mb,NaN,134.0,1.212,0.91,...,2018-10-27T18:48:13.227Z,"68km NE of Miyako, Japan",earthquake,7.4,6.4,0.041,192.0,reviewed,us,us
8,2018-10-27T10:39:04.280Z,18.3666,120.8819,10.00,4.6,mb,NaN,106.0,4.914,0.87,...,2018-10-27T12:29:00.280Z,"11km S of Adams, Philippines",earthquake,8.7,1.9,0.059,86.0,reviewed,us,us
9,2018-10-27T10:13:41.990Z,37.2921,20.5582,10.00,4.3,mb,NaN,56.0,2.489,0.72,...,2018-10-27T11:17:44.040Z,"53km SSW of Lithakia, Greece",earthquake,2.6,1.9,0.123,19.0,reviewed,us,us
